In [ ]:
import Pkg; Pkg.activate(".")

In [ ]:
using Revise, Graphs, Coarsening, MatrixMarket, Random, SparseArrays, GraphIO, DataStructures

In [ ]:
using LinearOrdering

In [ ]:
includet("./dynamictreewidth.jl")

In [ ]:
onesum = PSum(1)
twosum = PSum(2)

In [ ]:
config = (
    windowsizes= [5, 10, 15, 20, 25, 30],
    compat_sweeps=10,
    window_sweeps = 10,
    stride_percent=0.5,
    gauss_sweeps=10,
    coarsening=VolumeCoarsening(0.4, 2.0, 1),
    coarsest=10, 
    pad_percent=0.05, 
    node_window_size=10,
    seed = 20
)

In [ ]:
onesumgraphs = (
    grid4 = grid([4, 4]),
    grid33 = grid([33, 33]),
    mesh100 = grid([100, 100]),
    bintree10 = binary_tree(10)
)
onesumoptima = (
    grid33 = 31680,
    mesh100 = 868820,
    bintree10 = 3696
)
twosumgraphs = (
    bintree10 = binary_tree(10),
    jagmesh = SimpleGraph(mmread("./jagmesh9.mtx"))
)
twosumoptima = (
    bintree10 = 8.85 * 10 ^ 4,
    jagmesh=1.1 * 010 ^ 6
)

In [ ]:
jagA = mmread("./jagmesh9.mtx")
fix_adjacency!(jagA)
dropzeros!(jagA)
jag = SimpleGraph(jagA)
# G = SimpleGraph(loadgraph("./add20.rmf", "graph_key", EdgeListFormat()))
# G = grid([4, 4])
# G = SimpleGraph(makeadj(mmread("./graphs/regular3_16_2.mtx")))

In [ ]:
bus = mmread("./graphs/685_bus.mtx")
fix_adjacency!(bus)
dropzeros!(bus)
bus = SimpleGraph(bus)

In [ ]:
GDA = mmread("./graphs/GD96_d.mtx")
GDA = 1.0 * ((GDA .+ GDA') .!= 0)
fix_adjacency!(GDA)
dropzeros!(GDA)
GD = SimpleGraph(GDA)

In [ ]:
position_to_idx, idx_to_position = ordergraph(onesum, G; config...);

In [ ]:
LinearOrdering.evalorder(onesum, adjacency_matrix(G), idx_to_position)

In [ ]:
adjacency_matrix(G)[position_to_idx, position_to_idx]

# Issues Remaining
- Something is wrong with compatible relaxations & uncoarsening
- Diagnose why coarse edge weights < 1 for r > 1
- Window minimization rarely (if ever) accepts changes
- Test two level problem (size 20 graph)
    - Disable window min, use node by node
    - Test w/o smoothing or coarsening
- Why does it seem like our solutions are random at each level?

In [ ]:
using DataStructures

In [ ]:
includet("./dynamictreewidth.jl")

In [ ]:
import QXGraphDecompositions

In [ ]:
config = (
    windowsizes= [5, 10, 15, 20, 25, 30],
    compat_sweeps=10,
    window_sweeps = 10,
    stride_percent=0.5,
    gauss_sweeps=10,
    coarsening=VolumeCoarsening(0.4, 2.0, 1),
    coarsest=10, 
    pad_percent=0.05, 
    node_window_sweeps=10,
    node_window_size=10,
    seed = 11
)

In [ ]:
# circuitfiles = [
#         "./graphs/regular3_32_2.mtx",
#         "./graphs/regular3_32_3.mtx",
#         "./graphs/regular3_32_4.mtx",
#         "./graphs/regular3_32_5.mtx",
#         "./graphs/regular4_32_2.mtx",
#         "./graphs/regular4_32_3.mtx",
#         "./graphs/regular4_32_4.mtx",
#         "./graphs/regular4_32_5.mtx",
#         "./graphs/regular5_32_2.mtx",
#         "./graphs/regular5_32_3.mtx",
#         "./graphs/regular5_32_4.mtx",
#         "./graphs/regular5_32_5.mtx",
#         ]
circuitfiles = [
    "./graphs/regular$(d)_32_$(p)_$(seed).mtx" for d in 3:5 for p in 2:5 for seed in 0:9
];

In [ ]:
circuitadj = makeadj.(mmread.(circuitfiles));

In [ ]:
circuits = SimpleGraph.(circuitadj);

In [ ]:
check1, check2 = circuits[8], circuits[12]

In [ ]:
diameter(check1)

In [ ]:
diameter(check2)

In [ ]:
diameter(jag)

In [ ]:
diameter.(circuits)

In [ ]:
G = circuits[6]

In [ ]:
position_to_idx, idx_to_position = ordergraph(onesum, G; config...);

In [ ]:
using Logging

In [ ]:
# io = open("./tw_variance.csv", "w")
# write(io, "Name,V,E,Flowcutter,OneSumVal,TW\n")
results = []
# with_logger(ConsoleLogger(stderr, Logging.Debug)) do
    @showprogress 1 "Graph..." for (q, G) in enumerate(circuits)
        # fc = QXGraphDecompositions.flow_cutter(tolight((G)), 5)
        # fctw = fc[:treewidth]
        besttw = Inf
        bestonesumval = Inf
        @showprogress 1 "Order..." for r in 5
            @showprogress 1 "Seed..." for i in 1:20
                config = (
                    compat_sweeps=10,
                    stride_percent=0.5,
                    gauss_sweeps=10,
                    coarsening=VolumeCoarsening(0.4, 2.0, r),
                    coarsest=10, 
                    pad_percent=0.05, 
                    node_window_sweeps=10,
                    node_window_size=1,
                    seed = i
                )
                position_to_idx, idx_to_position = ordergraph(onesum, G; config...);
                onesumval = LinearOrdering.evalorder(onesum, adjacency_matrix(G), idx_to_position)
                # td = order_width(G, position_to_idx, idx_to_position);
                tw, _ = recursive_width(adjacency_matrix(G), position_to_idx, idx_to_position)
                if tw <= besttw
                    besttw = tw
                    bestonesumval = onesumval
                end
                # ok = check_td(vectorize_decomp(td)..., G)[1];
            end                

        end            
        # write(io, "$(circuitfiles[q]),$(nv(G)),$(ne(G)),$(fctw),$bestonesumval,$(besttw - 1)\n")
        # flush(io)
    end
# end
close(io)

In [ ]:
config = (
    compat_sweeps=10,
    stride_percent=0.5,
    gauss_sweeps=10,
    coarsening=VolumeCoarsening(0.4, 2.0, 5),
    coarsest=10, 
    pad_percent=0.05, 
    node_window_sweeps=10,
    node_window_size=1,
    seed = 1
)
position_to_idx, idx_to_position = ordergraph(onesum, bus; config...);

In [ ]:
adjacency_matrix(bus)[position_to_idx, position_to_idx]

In [ ]:
rem_vertex!(jag, 609)

In [ ]:
Revise.retry()

In [ ]:
rem_vertex!(jag, 1)
adjacency_matrix(jag)

In [ ]:
using Statistics

In [ ]:
minimum(results)

In [ ]:
median(results)

In [ ]:
std(results)

In [ ]:
maximum(results)

In [ ]:
argmax(results)

In [ ]:
results

In [ ]:
td = order_width(G, position_to_idx, idx_to_position);

In [ ]:
bags, tree = vectorize_decomp(td);

In [ ]:
maximum(length, bags)

In [ ]:
tree

In [ ]:
check_td(bags, tree, G)

In [ ]:
import QXGraphDecompositions

In [ ]:
teststst(circuitfiles)

In [ ]:
using ProgressMeter

In [ ]:
function teststst(files)
    println("Name | #V | #E | Diameter | FlowcutterTW | OneSumValue | OneSumTW | OneSumOK | TwoSumValue | TwoSumTW | TwoSumOK")
    @showprogress 1 "Computing..." for file in files
        B = mmread(file)
        testst(file, B)
    end
end

In [ ]:
function testst(file, B)
    circuit = SimpleGraph(makeadj(B))
    td = QXGraphDecompositions.flow_cutter(tolight((circuit)), 1)
    position_to_idx1, idx_to_position1 = ordergraph(onesum, circuit; config...);
    position_to_idx2, idx_to_position2 = ordergraph(twosum, circuit; config...);
    oneval = LinearOrdering.evalorder(onesum, adjacency_matrix(circuit), idx_to_position1)
    twoval = LinearOrdering.evalorder(twosum, adjacency_matrix(circuit), idx_to_position2)
    td1 = order_width((circuit), position_to_idx1, idx_to_position1);
    td2 = order_width((circuit), position_to_idx2, idx_to_position2);
    ok1 = check_td(vectorize_decomp(td1)..., circuit)[1];
    ok2 = check_td(vectorize_decomp(td2)..., circuit)[1];
    println("$file | $(nv(circuit)) | $(ne(circuit)) | $(diameter(circuit)) | $(td[:treewidth]) | $oneval | $(td1.width - 1) | $ok1 | $twoval | $(td2.width - 1) | $ok2")
end

In [ ]:
td = QXGraphDecompositions.flow_cutter(tolight((circuit)), 10);

In [ ]:
td[:treewidth]

In [ ]:
compare([circuit])

In [ ]:
using DataFrames, Plots, CSV, StatsPlots

In [ ]:
df = DataFrame(CSV.File("./tw_variance.csv"));

In [ ]:
names(df)

In [ ]:
gdf = groupby(df, :Name);

In [ ]:
df

In [ ]:
cgdf = combine(gdf, :Flowcutter => minimum, :TW=> minimum)

In [ ]:
@df cgdf groupedbar([:Flowcutter_minimum :TW_minimum], bar_position =:dodge, xlabel="Graphs", ylabel="Width", bar_width=0.4)

In [ ]:
for subdf in groupby(df, [:Name, :Order])
    l = @layout [a b]
    p1 = @df subdf histogram(:OneSumVal, legend=false)
    p2 = @df subdf bar(
        unique(:TW), 
        [count(==(e), :TW) for e in unique(:TW)];
        legend = false, xlabel="One Sum Value"
    )
    @df subdf bar!(p2, 
        [minimum(:Flowcutter)], [count(==(minimum(:Flowcutter)), :TW)];
        color = :red, legend=false, xlabel = "Order Width", title="r = $(subdf.Order[1])"
    )
    display(plot(p1, p2; layout = l, size=(700, 200)))
end

In [ ]:
for subdf in gdf
    display(@df subdf scatter(:OneSumVal, :TW; size=(300, 300), legend=false, title=subdf.Name[1]))
end

In [ ]:
function write_gr(filepath, graph)
    io = open(filepath, "w")
    write(io, "p tw $(nv(graph)) $(ne(graph))\n")
    for e in edges(graph)
        write(io, "$(e.src) $(e.dst)\n")
    end
    close(io)
end

In [ ]:
for (name, graph) in Iterators.zip(circuitfiles, circuits)
    path, _ = splitext(name)
    path = path * ".gr"
    line = linegraph(graph)
    write_gr(path, line)
end

In [ ]:
tam_log = readlines("./tamaki_widths.log");

In [ ]:
count(startswith.(tam_log, '/'))

In [ ]:
using DataFrames

In [ ]:
idx = 1
rows = []
parseint(s) = parse(Int, join(collect(Iterators.filter(isnumeric, s))))
while idx < length(tam_log)
    name = tam_log[idx]
    idx += 2
    while idx <= length(tam_log) && startswith(tam_log[idx], 'c')
        row = (name, parseint(tam_log[idx]), parseint(tam_log[idx+1]))
        push!(rows, row)
        idx += 2
    end
end
tamaki_df = DataFrame(rows);

In [ ]:
getwidthsdf(df, s) = combine(groupby(filter(:3 => x -> x < s, df), :1), :2 => minimum)[!, Symbol("2_minimum")];

In [ ]:
for v in tw5s
    println(v)
end

In [ ]:
using TickTock

In [ ]:
rowscost = []
@showprogress 1 "Graph..." for (name, G) in Iterators.zip(circuitfiles, circuits)
    bestcost = Inf
    tick()
    @showprogress 1 "Seed..." for i in 1:50
        config = (
            compat_sweeps=10,
            stride_percent=0.5,
            gauss_sweeps=10,
            coarsening=VolumeCoarsening(0.4, 2.0, 5),
            coarsest=10, 
            pad_percent=0.05, 
            node_window_sweeps=10,
            node_window_size=1,
            seed = i
        )
        position_to_idx, idx_to_position = ordergraph(onesum, G; config...);
        onesumval = LinearOrdering.evalorder(onesum, adjacency_matrix(G), idx_to_position)
        cost, _ = recursive_width(adjacency_matrix(G), position_to_idx, idx_to_position; flops=true)
        t = peektimer()
        if cost <= bestcost
            bestcost = cost
            push!(rowscost, (name, bestcost, t))
        end
        if t >= 10
            break
        end
    end    
    tok()
end

In [ ]:
using DataFrames

In [ ]:
order_df = DataFrame(rowscost)

In [ ]:
order_df = DataFrame(rowscw)

In [ ]:
for v in getwidthsdf(order_df, 10)
    println(v)
end

In [ ]:
path = [(116, 141),
 (176, 190),
 (50, 84),
 (18, 188),
 (186, 187),
 (49, 76),
 (18, 185),
 (104, 133),
 (168, 183),
 (181, 182),
 (180, 181),
 (52, 88),
 (22, 179),
 (115, 124),
 (167, 177),
 (175, 176),
 (94, 130),
 (157, 174),
 (43, 65),
 (14, 172),
 (170, 171),
 (169, 170),
 (168, 169),
 (41, 57),
 (13, 167),
 (47, 54),
 (20, 165),
 (163, 164),
 (39, 77),
 (13, 162),
 (160, 161),
 (102, 116),
 (143, 159),
 (76, 115),
 (140, 157),
 (155, 156),
 (75, 107),
 (144, 154),
 (152, 153),
 (151, 152),
 (150, 151),
 (99, 120),
 (127, 149),
 (82, 102),
 (141, 147),
 (145, 146),
 (46, 57),
 (21, 144),
 (28, 71),
 (4, 142),
 (140, 141),
 (139, 140),
 (138, 139),
 (31, 68),
 (8, 137),
 (24, 46),
 (2, 135),
 (133, 134),
 (65, 106),
 (109, 132),
 (84, 99),
 (112, 130),
 (128, 129),
 (127, 128),
 (126, 127),
 (65, 103),
 (104, 125),
 (22, 44),
 (1, 123),
 (121, 122),
 (120, 121),
 (73, 97),
 (100, 119),
 (64, 95),
 (98, 117),
 (115, 116),
 (22, 53),
 (2, 114),
 (21, 42),
 (2, 112),
 (110, 111),
 (109, 110),
 (108, 109),
 (28, 43),
 (10, 107),
 (20, 41),
 (3, 105),
 (103, 104),
 (19, 38),
 (3, 102),
 (51, 76),
 (83, 100),
 (98, 99),
 (97, 98),
 (51, 69),
 (86, 96),
 (51, 73),
 (78, 94),
 (92, 93),
 (91, 92),
 (51, 74),
 (73, 90),
 (0, 15),
 (34, 88),
 (86, 87),
 (85, 86),
 (46, 57),
 (80, 84),
 (26, 32),
 (12, 82),
 (80, 81),
 (48, 63),
 (64, 79),
 (14, 35),
 (1, 77),
 (75, 76),
 (74, 75),
 (13, 24),
 (1, 73),
 (71, 72),
 (44, 54),
 (57, 70),
 (33, 53),
 (54, 68),
 (66, 67),
 (65, 66),
 (64, 65),
 (12, 32),
 (1, 63),
 (11, 29),
 (1, 61),
 (59, 60),
 (36, 45),
 (46, 58),
 (56, 57),
 (55, 56),
 (54, 55),
 (15, 24),
 (6, 53),
 (30, 35),
 (46, 51),
 (49, 50),
 (30, 34),
 (43, 48),
 (13, 22),
 (5, 46),
 (44, 45),
 (43, 44),
 (42, 43),
 (11, 15),
 (4, 41),
 (7, 12),
 (1, 39),
 (37, 38),
 (5, 14),
 (0, 36),
 (4, 8),
 (0, 34),
 (32, 33),
 (31, 32),
 (10, 18),
 (25, 30),
 (14, 20),
 (19, 28),
 (26, 27),
 (10, 17),
 (18, 25),
 (9, 16),
 (15, 23),
 (21, 22),
 (20, 21),
 (19, 20),
 (18, 19),
 (11, 14),
 (13, 17),
 (9, 11),
 (12, 15),
 (13, 14),
 (5, 6),
 (2, 12),
 (2, 5),
 (0, 10),
 (8, 9),
 (7, 8),
 (6, 7),
 (3, 4),
 (3, 5),
 (1, 2),
 (0, 3),
 (1, 2),
 (0, 1)];